#Câu 1: Classification - Women's E-Commerce Clothing Reviews (1.0 điểm)
Use **womens-ecommerce-clothing-reviews** dataset (file 
Womens_Clothing_E_Commerce_Reviews.xlsx, sheetName: **Reviews**) to build a 
model to predict products’ ratings (based on **Review Text** and other optional 
features)
<br>Please predict ratings for products in Womens_Clothing_E_Commerce_Reviews.xlsx, 
<br>sheetName: **new_reviews**.
<br>Read more information here:
https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

In [ ]:
#@title Install Pakages
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark  # Tìm Spark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,431 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6

In [ ]:
#@title Mount to drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/LDS9_K269_ONLINE_NgoThiNga/LDS9_K269_NgoThiNga_Cuoi_ky/Du lieu cung cap/womens-ecommerce-clothing-reviews'

/content/gdrive/MyDrive/LDS9_K269_ONLINE_NgoThiNga/LDS9_K269_NgoThiNga_Cuoi_ky/Du lieu cung cap/womens-ecommerce-clothing-reviews


In [ ]:
#@title Import pakages
import pyspark
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, explode
import pandas as pd
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.ml.feature import Binarizer,Bucketizer, OneHotEncoder, StringIndexer, Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.sql.functions import isnan, count, regexp_extract, col, datediff, when, length, log, array, lit, trim

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, DecisionTreeClassificationModel
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel, GBTClassifier
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml.classification import NaiveBayes, NaiveBayesModel
from pyspark.ml.classification import LinearSVC, LinearSVCModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [ ]:
sc = SparkContext()
spark = SparkSession.builder.appName('cau1').getOrCreate()

In [ ]:
df = pd.read_excel("Womens_Clothing_E_Commerce_Reviews.xlsx", index_col=0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23481 entries, 0 to 23480
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Clothing ID              23481 non-null  int64 
 1   Age                      23481 non-null  int64 
 2   Title                    19671 non-null  object
 3   Review Text              22636 non-null  object
 4   Rating                   23481 non-null  int64 
 5   Recommended IND          23481 non-null  int64 
 6   Positive Feedback Count  23481 non-null  int64 
 7   Division Name            23467 non-null  object
 8   Department Name          23467 non-null  object
 9   Class Name               23467 non-null  object
dtypes: int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
df.dropna(subset=['Review Text'], axis=0, inplace=True)

In [ ]:
#@title Create Spark DataFrame
schema = StructType([
                     StructField("Clothing ID", IntegerType(), True),
                     StructField("Age", IntegerType(), True),
                     StructField("Title", StringType(), True),
                     StructField("Review Text", StringType(), True),
                     StructField("Rating", IntegerType(), True),
                     StructField("Recommended IND", IntegerType(), True),
                     StructField("Positive Feedback Count", IntegerType(), True),
                     StructField("Division Name", StringType(), True),
                     StructField("Department Name", StringType(), True),
                     StructField("Class Name", StringType(), True),
                     
])

data = spark.createDataFrame(df, schema = schema)
data.show(5, False)

+-----------+---+-----------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+---------------+-----------------------+--------------+---------------+----------+
|Clothing ID|Age|Title                  |Review Text                                                                                                                                                                                                                                                                                                                            

In [ ]:
data.count()

22636

In [ ]:
data.printSchema()

root
 |-- Clothing ID: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Review Text: string (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Recommended IND: integer (nullable = true)
 |-- Positive Feedback Count: integer (nullable = true)
 |-- Division Name: string (nullable = true)
 |-- Department Name: string (nullable = true)
 |-- Class Name: string (nullable = true)



In [ ]:
# Kiểm tra dữ liệu nan
data.select([count(when(isnan(c), c)).alias(c) for c in data.columns]).toPandas().T


,0
Clothing ID,0
Age,0
Title,2966
Review Text,0
Rating,0
Recommended IND,0
Positive Feedback Count,0
Division Name,13
Department Name,13
Class Name,13


In [ ]:
# def to_null(c):
#     return when(~(col(c).isNull() | isnan(col(c)) | (trim(col(c)) == " ")), col(c))


# data.select(to_null('Review Text').alias('Review Text_notnull')).na.drop().show()

In [ ]:
data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).toPandas().T

,0
Clothing ID,0
Age,0
Title,0
Review Text,0
Rating,0
Recommended IND,0
Positive Feedback Count,0
Division Name,0
Department Name,0
Class Name,0


In [ ]:
data = data.withColumn('class', when(data.Rating > 4, 'like'). when(data.Rating <=2,  'Not_like').otherwise("neutral"))

In [ ]:
# Create length column
data = data.withColumn('length', length(data['Review Text']))
data = data.withColumn('log_Length', log(col('length')))
data = data.select("Review Text", "Rating", "class",'Age', 'log_Length','Positive Feedback Count')
data.show(3)

+--------------------+------+-------+---+-----------------+-----------------------+
|         Review Text|Rating|  class|Age|       log_Length|Positive Feedback Count|
+--------------------+------+-------+---+-----------------+-----------------------+
|Absolutely wonder...|     4|neutral| 33|3.970291913552122|                      0|
|Love this dress! ...|     5|   like| 34|5.713732805509369|                      4|
|I had such high h...|     3|neutral| 60|6.214608098422191|                      0|
+--------------------+------+-------+---+-----------------+-----------------------+
only showing top 3 rows



In [ ]:
data.groupby('class').mean('log_Length','Positive Feedback Count').show()

+--------+------------------+----------------------------+
|   class|   avg(log_Length)|avg(Positive Feedback Count)|
+--------+------------------+----------------------------+
| neutral|5.6490521610621585|          2.7464096260835813|
|    like| 5.541685470992448|          2.4083260228088363|
|Not_like| 5.620096081190402|          3.4239864864864864|
+--------+------------------+----------------------------+



In [ ]:
data.groupby('class').count().show()

+--------+-----+
|   class|count|
+--------+-----+
| neutral| 7729|
|    like|12539|
|Not_like| 2368|
+--------+-----+



## Feature Transformation

In [ ]:
indexer = StringIndexer(inputCol='class', outputCol='label')
# Indexer identifies categories in the data
indexer_model = indexer.fit(data)
# Indexer creates a new column with numeric index values
data_final = indexer_model.transform(data)
data_final.groupBy(['label','class']).count().show(3)

+-----+--------+-----+
|label|   class|count|
+-----+--------+-----+
|  0.0|    like|12539|
|  1.0| neutral| 7729|
|  2.0|Not_like| 2368|
+-----+--------+-----+



In [ ]:
data_final.show(3)

+--------------------+------+-------+---+-----------------+-----------------------+-----+
|         Review Text|Rating|  class|Age|       log_Length|Positive Feedback Count|label|
+--------------------+------+-------+---+-----------------+-----------------------+-----+
|Absolutely wonder...|     4|neutral| 33|3.970291913552122|                      0|  1.0|
|Love this dress! ...|     5|   like| 34|5.713732805509369|                      4|  0.0|
|I had such high h...|     3|neutral| 60|6.214608098422191|                      0|  1.0|
+--------------------+------+-------+---+-----------------+-----------------------+-----+
only showing top 3 rows



In [ ]:
# Chuẩn hóa dữ liệu 'Review Text'
tokenizer = Tokenizer(inputCol='Review Text', outputCol='token_text')
stopremove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='cv')
idf = IDF(inputCol='cv', outputCol='tf-idf')
#class_to_num = StringIndexer(inputCol='class', outputCol='label')

In [ ]:
assembler = VectorAssembler(inputCols=['tf-idf','Age','log_Length','Positive Feedback Count'],
                           outputCol='features')

## Tạo tập dữ liệu Train và Test

In [ ]:
data_final = data_final.select("Review Text",'Age', 'log_Length','Positive Feedback Count', 'label')

In [ ]:
training, testing = data_final.randomSplit([0.8,0.2])

In [ ]:
#training.groupBy('label').count().show()

In [ ]:
# testing.groupBy('label').count().show()

## Áp dụng các mô hình Classification
+ Logistic Regression
+ Naive Bayes
+ RandomForest
+ DecisionTree


In [ ]:
# init classification model
lg = LogisticRegression(featuresCol = "features",
                      labelCol = "label", 
                      maxIter = 20, 
                      regParam=0.3, 
                      predictionCol = "prediction")
nb = NaiveBayes(featuresCol = "features",
                smoothing=1.0,
                modelType="multinomial",
                labelCol = "label",
                predictionCol = "prediction")
rf = RandomForestClassifier(featuresCol = "features", 
                      labelCol = "label",
                      numTrees=500,
                      maxDepth = 5,
                      maxBins = 64,
                      predictionCol = "prediction")
ds = DecisionTreeClassifier(featuresCol = "features",
                      labelCol = "label",
                      predictionCol = "prediction")

In [ ]:
# init pipeline for models
# logisticRegression pipeline
pipeline_lg = Pipeline(stages=[tokenizer,
                            stopremove,
                            count_vec,
                            idf,
                            assembler, lg])

# Naive Bayes pipeline
pipeline_nb = Pipeline(stages=[tokenizer,
                            stopremove,
                            count_vec,
                            idf,
                            assembler, nb])

# RandomForest pipeline
pipeline_rf = Pipeline(stages=[tokenizer,
                            stopremove,
                            count_vec,
                            idf,
                            assembler, rf])

# DecisionTree pipeline
pipeline_ds = Pipeline(stages=[tokenizer,
                            stopremove,
                            count_vec,
                            idf,
                            assembler, ds])

In [ ]:
# fit pipeline
lg_model = pipeline_lg.fit(training)
nb_model = pipeline_nb.fit(training)
rf_model = pipeline_rf.fit(training)
ds_model = pipeline_ds.fit(training)

In [ ]:
#apply transform
test_result_lg = lg_model.transform(testing)
test_result_nb = nb_model.transform(testing)
test_result_rf = rf_model.transform(testing)
test_result_ds = ds_model.transform(testing)

In [ ]:
# avaluate models
acc_eval = MulticlassClassificationEvaluator(labelCol="label",
                                        predictionCol="prediction",
                                        metricName="accuracy")
acc_lg = acc_eval.evaluate(test_result_lg)
print("Accuracy of Logistic model at predicting product's rating: {0:2.2f}%".format(acc_lg*100))
print('-'*80)
acc_nb = acc_eval.evaluate(test_result_nb)
print("Accuracy of Naive Bayes model at predicting product's rating: {0:2.2f}%".format(acc_nb*100))
print('-'*80)
acc_rf = acc_eval.evaluate(test_result_rf)
print("Accuracy of Random Forest model at predicting product's rating: {0:2.2f}%".format(acc_rf*100))
print('-'*80)
acc_ds = acc_eval.evaluate(test_result_ds)
print("Accuracy of Decision Tree model at predicting product's rating: {0:2.2f}%".format(acc_ds*100))


Accuracy of Logistic model at predicting product's rating: 67.87%
--------------------------------------------------------------------------------
Accuracy of Naive Bayes model at predicting product's rating: 66.31%
--------------------------------------------------------------------------------
Accuracy of Random Forest model at predicting product's rating: 55.54%
--------------------------------------------------------------------------------
Accuracy of Decision Tree model at predicting product's rating: 57.47%


**Lựa chọn LogisticRegression model**

In [ ]:
# Create a confusion matrix
test_result_lg.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       0.0|  122|
|  1.0|       1.0|  804|
|  0.0|       1.0|  294|
|  1.0|       0.0|  721|
|  2.0|       2.0|   62|
|  2.0|       1.0|  294|
|  1.0|       2.0|   46|
|  0.0|       0.0| 2262|
|  0.0|       2.0|    4|
+-----+----------+-----+



**Nhận xét:**
+ Accuracy trên các model còn thấp
+ Cần cân bằng lại dữ liệu, để cải thiện model

## Cân bằng lại dữ liệu trên tập training


In [ ]:
like_df = training.filter(col("label") == 0.0) 
neutral_df = training.filter(col("label") == 1.0) 
not_like_df = training.filter(col("label") == 2.0) 
ratio_1 = int(like_df.count()/neutral_df.count()) 
ratio_2 = int(like_df.count()/not_like_df.count()) 
print("ratio like/neutral: {}".format(ratio_1)) 
print("ratio like/not_like: {}".format(ratio_2))

ratio like/neutral: 1
ratio like/not_like: 5


In [ ]:
# Resample neutral
a1 = range(ratio_1)
#duplicate the minority rows
oversampled_neutral_df = neutral_df.withColumn('dummy',
                                              explode(array([lit(x) for x in a1]))).drop('dummy')
# combine both oversampled minority rows and previous majority rows
traning_resample = like_df.unionAll(oversampled_neutral_df)
traning_resample.show(5)

+--------------------+---+-----------------+-----------------------+-----+
|         Review Text|Age|       log_Length|Positive Feedback Count|label|
+--------------------+---+-----------------+-----------------------+-----+
|-like others, i a...| 34|5.541263545158426|                      1|  0.0|
|...on this great ...| 37| 4.48863636973214|                      2|  0.0|
|1. i'm 5'5" tall,...| 57|4.382026634673881|                      5|  0.0|
|1. i'm 5'5" tall,...| 57|6.206575926724928|                     19|  0.0|
|1. i'm 5'5" tall,...| 57|6.214608098422191|                     19|  0.0|
+--------------------+---+-----------------+-----------------------+-----+
only showing top 5 rows



In [ ]:
traning_resample.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 9979|
|  1.0| 6158|
+-----+-----+



In [ ]:
# Resample not_like
a2 = range(ratio_2)
#duplicate the minority rows
oversampled_notlike_df = not_like_df.withColumn('dummy',
                                              explode(array([lit(x) for x in a2]))).drop('dummy')
# combine both oversampled minority rows and previous majority rows
traning_resample = traning_resample.unionAll(oversampled_notlike_df)
traning_resample.show(5)

+--------------------+---+-----------------+-----------------------+-----+
|         Review Text|Age|       log_Length|Positive Feedback Count|label|
+--------------------+---+-----------------+-----------------------+-----+
|-like others, i a...| 34|5.541263545158426|                      1|  0.0|
|...on this great ...| 37| 4.48863636973214|                      2|  0.0|
|1. i'm 5'5" tall,...| 57|4.382026634673881|                      5|  0.0|
|1. i'm 5'5" tall,...| 57|6.206575926724928|                     19|  0.0|
|1. i'm 5'5" tall,...| 57|6.214608098422191|                     19|  0.0|
+--------------------+---+-----------------+-----------------------+-----+
only showing top 5 rows



In [ ]:
traning_resample.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 9979|
|  1.0| 6158|
|  2.0| 9450|
+-----+-----+



**Áp dụng model cho dữ liệu sau khi cân bằng**

In [ ]:
# Fit models on resampled training set
lg_model_2 = pipeline_lg.fit(traning_resample)
nb_model_2 = pipeline_nb.fit(traning_resample)
rf_model_2 = pipeline_rf.fit(traning_resample)
ds_model_2 = pipeline_ds.fit(traning_resample)

In [ ]:
# apply transform
test_result_lg_2 = lg_model_2.transform(testing)
test_result_nb_2 = nb_model_2.transform(testing)
test_result_rf_2 = rf_model_2.transform(testing)
test_result_ds_2 = ds_model_2.transform(testing)

In [ ]:
# avaluate models
acc_eval = MulticlassClassificationEvaluator(labelCol="label",
                                        predictionCol="prediction",
                                        metricName="accuracy")
acc_lg_2 = acc_eval.evaluate(test_result_lg_2)
print("Accuracy of Logistic model at predicting product's rating: {0:2.2f}%".format(acc_lg_2*100))
print('-'*80)
acc_nb_2 = acc_eval.evaluate(test_result_nb_2)
print("Accuracy of Naive Bayes model at predicting product's rating: {0:2.2f}%".format(acc_nb_2*100))
print('-'*80)
acc_rf_2 = acc_eval.evaluate(test_result_rf_2)
print("Accuracy of Random Forest model at predicting product's rating: {0:2.2f}%".format(acc_rf_2*100))
print('-'*80)
acc_ds_2 = acc_eval.evaluate(test_result_ds_2)
print("Accuracy of Decision Tree model at predicting product's rating: {0:2.2f}%".format(acc_ds_2*100))


Accuracy of Logistic model at predicting product's rating: 66.57%
--------------------------------------------------------------------------------
Accuracy of Naive Bayes model at predicting product's rating: 64.92%
--------------------------------------------------------------------------------
Accuracy of Random Forest model at predicting product's rating: 60.38%
--------------------------------------------------------------------------------
Accuracy of Decision Tree model at predicting product's rating: 47.04%


**Lựa chọn LogisticRegression model**

In [ ]:
# Create a confusion matrix
test_result_lg_2.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       0.0|   65|
|  1.0|       1.0|  536|
|  0.0|       1.0|  271|
|  1.0|       0.0|  653|
|  2.0|       2.0|  304|
|  2.0|       1.0|  109|
|  1.0|       2.0|  382|
|  0.0|       0.0| 2228|
|  0.0|       2.0|   61|
+-----+----------+-----+



**Nhận xét:**
+ Sau khi resample lại các class, độ chính xác của các model không được cải thiện.
+ Tuy nhiên model 2 dự đoán tốt hơn trên các class 0, 1, 2 so với model 1 khi chưa cân bằng dữ liệu. Vì vậy ta chọn model 2 cho việc dự đoán trên tập dữ liệu mới.

## Lựa chọn model
+ LogisticRegression model có độ chính xác 66.57%, cao nhất trong các model đã sử dụng
+ Chọn LogisticRegression để  dự đoán

In [ ]:
test_result_lg_2.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 2560|
|  1.0| 1571|
|  2.0|  478|
+-----+-----+



In [ ]:
print('Accuracy of model: ', round(acc_lg_2*100,2), '%')

Accuracy of model:  66.57 %


## Xử lý dữ liệu từ sheet "new_reviews" và áp dụng model để dự đoán trên dữ liệu mới

**Tiền xử lý "new_reviews"**

In [ ]:
df_new = pd.read_excel("Womens_Clothing_E_Commerce_Reviews.xlsx", index_col=0, sheet_name='new_reviews')

In [ ]:
schema = StructType([
                     StructField("Clothing ID", IntegerType(), True),
                     StructField("Age", IntegerType(), True),
                     StructField("Title", StringType(), True),
                     StructField("Review Text", StringType(), True),
                     StructField("Recommended IND", IntegerType(), True),
                     StructField("Positive Feedback Count", IntegerType(), True),
                     StructField("Division Name", StringType(), True),
                     StructField("Department Name", StringType(), True),
                     StructField("Class Name", StringType(), True),
                     
])
data_new = spark.createDataFrame(df_new, schema = schema)

In [ ]:
data_new.show()

+-----------+---+--------------------+--------------------+---------------+-----------------------+-------------+---------------+----------+
|Clothing ID|Age|               Title|         Review Text|Recommended IND|Positive Feedback Count|Division Name|Department Name|Class Name|
+-----------+---+--------------------+--------------------+---------------+-----------------------+-------------+---------------+----------+
|       1077| 53|Dress looks like ...|Dress runs small ...|              0|                     14|      General|        Dresses|   Dresses|
|        862| 66|            Cute top|Nice top. armhole...|              1|                      2|      General|           Tops|     Knits|
|       1080| 31|        Underwhelmed|Was really excite...|              0|                      1|      General|        Dresses|   Dresses|
|        936| 35|  Absolutely perfect|If you are going ...|              0|                      9|      General|           Tops|  Sweaters|
|        872|

In [ ]:
# Kiểm tra dữ liệu null
data_new.select([count(when(isnan(c), c)).alias(c) for c in data_new.columns]).toPandas().T


,0
Clothing ID,0
Age,0
Title,0
Review Text,0
Recommended IND,0
Positive Feedback Count,0
Division Name,0
Department Name,0
Class Name,0


In [ ]:
# => Không có dữ liệu nan

In [ ]:
data_new.select([count(when(col(c).isNull(), c)).alias(c) for c in data_new.columns]).toPandas().T

,0
Clothing ID,0
Age,0
Title,0
Review Text,0
Recommended IND,0
Positive Feedback Count,0
Division Name,0
Department Name,0
Class Name,0


In [ ]:
# => không có dữ liệu null

In [ ]:
# Create length column
data_new = data_new.withColumn('length', length(data_new['Review Text']))
data_new = data_new.withColumn('log_Length', log(col('length')))
data_new.show()

+--------------------+---+-----------------+-----------------------+------+
|         Review Text|Age|       log_Length|Positive Feedback Count|length|
+--------------------+---+-----------------+-----------------------+------+
|Dress runs small ...| 53|5.817111159963204|                     14|   336|
|Nice top. armhole...| 66|6.104793232414985|                      2|   448|
|Was really excite...| 31|5.998936561946683|                      1|   403|
|If you are going ...| 35|5.786897381366708|                      9|   326|
|I saw this online...| 35|5.707110264748875|                      0|   301|
+--------------------+---+-----------------+-----------------------+------+



In [ ]:
# Lựa chọn thuộc tính phù hợp với tập đã train
data_new = data_new.select("Review Text", 'Age', 'log_Length','Positive Feedback Count')
data_new.show(3)

+--------------------+---+-----------------+-----------------------+
|         Review Text|Age|       log_Length|Positive Feedback Count|
+--------------------+---+-----------------+-----------------------+
|Dress runs small ...| 53|5.817111159963204|                     14|
|Nice top. armhole...| 66|6.104793232414985|                      2|
|Was really excite...| 31|5.998936561946683|                      1|
+--------------------+---+-----------------+-----------------------+
only showing top 3 rows



In [ ]:
# Check dataset with LogisticRegression model
predictions = lg_model_2.transform(data_new)
predictions.select('Review Text','probability','prediction').show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------+----------+
|Review Text                                                                                                                                                                                                                                                                                                                                                                                                                                                     |probability                

## Lưu và Tải model

In [ ]:
lg_model_2.save('/content/gdrive/MyDrive/LDS9_K269_ONLINE_NgoThiNga/LDS9_K269_NgoThiNga_Cuoi_ky/Cau1_model')

In [ ]:
from pyspark.ml import PipelineModel
lg_model_load = PipelineModel.load('Cau1_model')